## 1. Extraccion de Datos

### 1.1 Obtencion de datos Proprocesados

In [0]:
import pandas as pd
from io import BytesIO
from minio import Minio
import urllib3
# Establecer el número máximo de columnas a mostrar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Conectar al servidor de MinIO
client = Minio(
    #"minio.minio-user.svc.cluster.local:80",  # Dirección del servidor MinIO
    "192.168.25.223",  # Dirección del servidor MinIO
    access_key="c9iXL6uoEu8r35odfMLV",         # Clave de acceso
    secret_key="r3Wx21EmA41gB3mH65mvBVG9sH3lIMPwSmD0WMtI",  # Clave secreta
    secure=False,  # Cambia a True si usas HTTPS
    http_client=urllib3.PoolManager(cert_reqs='CERT_NONE')  # Ignora la verificación del certificado
)

# Nombre del bucket y del archivo
bucket_name = 'hudbayprocessed'
object_name = 'proyectocombustible_processed/table_delta_28.parquet'  # Nombre del archivo Parquet que quieres leer

# Obtener el archivo Parquet del bucket
response = client.get_object(bucket_name, object_name)
data = response.read()

# Convertir los datos leídos en un DataFrame de Pandas
parquet_buffer = BytesIO(data)
datos = pd.read_parquet(parquet_buffer)

# Mostrar el DataFrame
datos.head()

In [0]:
import pyarrow.parquet as pq

ruta_carpeta_parquet = "/dbfs/mnt/datalakemlopsd4m/processed/proyectocombustible_processed/hudbay/table_delta_28.parquet"
tabla = pq.read_table(ruta_carpeta_parquet)

# Convertir a DataFrame de pandas y mostrar las primeras filas
datos = tabla.to_pandas()
datos.head(1)

### 1.2 Revision de los datos Extraidos

In [0]:
datos.shape

Verificar si hay valores Nulos

In [0]:
datos.isnull().sum()

In [0]:
#Ver la fecha minima y maxima
datos['instant_date_t'].min(), datos['instant_date_t'].max(),

In [0]:
#ve los nombres de las columnas
datos.columns

In [0]:
# #2. Ajuste de los datos a Series Temporales
# #2.1 Se establece la columna 'Time' como el índice del DataFrame "datos"
datos = datos.set_index('instant_date_t')

# #2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos.index.to_series().diff().dt.total_seconds()
df_time_diffs.value_counts()

## 2.  Preparacion de datos para Analitica Predictiva

### 2. Limpieza de valores Nulos

In [0]:
# 3.1 Supongamos que tienes un DataFrame llamado datos
valores_nulos = datos.isnull().sum()
valores_nulos_ordenados = valores_nulos.sort_values(ascending=False)
porcentaje_nulos = (valores_nulos_ordenados / len(datos)) * 100
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > 40].index
datos = datos.drop(columnas_a_eliminar, axis=1)

In [0]:
porcentaje_nulos

In [0]:
columnas_a_eliminar

In [0]:
datos.shape

In [0]:
datos['nombre'].head()

In [0]:
datos.dtypes

In [0]:
#4.1 Eliminando columnas especificas que no aportan informacion ( # errors='ignore':ignore cualquier error si alguna de las columnas especificadas no se encuentra en el DataFrame.)
datos = datos.drop(['equipment','nombre'], axis=1, errors='ignore') 

In [0]:
datos.shape

In [0]:
datos.isnull().sum()

In [0]:
# 3.4 Rellenar los valores nulos con ceros en todo el DataFrame
datos = datos.fillna(0)

In [0]:
datos.isnull().sum()

## 2.1 Tratamiento de Variables Categoricas

### 2.1.1 Metodo 1 - Usar un bucle para realizar el codigo de One Hot Encoding

2.1.1.1 VARIABLE Status Truck

In [0]:
import pandas as pd
# Obtener las categorías únicas
categorias = datos['Status Truck'].unique()

# Crear un DataFrame vacío con columnas para cada categoría, Diccionario one_hot_dict : categoria: clave es una categoría única de #nombre_equipo_carguio, y el [0]*len(datos) : valor asociado a cada clave es una lista de ceros de la misma longitud que el DataFrame 
one_hot_dict = {categoria: [0]*len(datos) for categoria in categorias}

# Llenar el diccionario con 1s en las posiciones correspondientes
# idx es el índice de la fila, val es el valor de nombre_equipo_carguio en esa fila
# Si idx=0 y val='CF02', entonces one_hot_dict['CF02'][0] = 1
for idx, val in datos['Status Truck'].items():
    one_hot_dict[val][idx] = 1

# Convertir el diccionario a un DataFrame
one_hot_df = pd.DataFrame(one_hot_dict)

#Unir el DataFrame original con el codificado
datos = pd.concat([datos, one_hot_df], axis=1)

# # Mostrar el resultado
datos[['Status Truck','Cargando', 'Viaje Descargado', 'Viaje Cargado', 'Descarga']].head()

### 2.2 Convertir la columna de tiempo a índice temporal si aún no lo es y Definimos Year, Month, week apartir de la fecha

In [0]:
import pandas as pd

# Convertir la columna de tiempo a índice temporal si aún no lo es
datos.set_index('instant_date_t', inplace=True)

# Crear nuevas columnas basadas en el índice temporal
datos['year'] = datos.index.year  # year
datos['month'] = datos.index.month # mes
datos['day'] = datos.index.day  # Día del mes (1 al 31 del mes)
datos['hour'] = datos.index.hour  # hora
datos['day_week_str'] = datos.index.strftime('%a')  # Día de la semana como string
datos['year_month'] = datos.index.strftime('%Y_%m')  # Formato 'Año_Mes'
datos.head()
# (Opcional) Si quieres devolver 'instant_date_t' como columna
#datos = datos.reset_index()

## 3 Filtro de datos a usar: Equipo de Acarreo a Analizar

In [0]:
# datos_equipo_28 = datos[datos['eq_id'] == 28]

In [0]:
datos_equipo_28.index.min()

In [0]:
# Filtrar los datos para el mes de enero de 2024 usando el índice 'instant_date_t'
datos_equipo_28_enero = datos_equipo_28[(datos_equipo_28.index >= datos_equipo_28.index.min() ) & (datos_equipo_28.index < '2024-02-01')]

# Mostrar los primeros registros filtrados
datos_equipo_28_enero.head()

In [0]:
datos_equipo_28_enero.shape

## 4. Seleccion de Variables para Modelos de Machine Learning

### 4.1 Seleccion variables con StatModels, para regresion lineal (variables significativas estadisticamente basado en la Hipotesis Nula)

In [0]:
import pandas as pd
import statsmodels.api as sm

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
X = datos_equipo_32[['x', 'y', 'z', 
        #'eq_id', 
       'id_trabaj_t',
       'tramosidsnew_t', 
        #'Status Truck', '
        'Left Rear Axle Rpm',
       'Right Rear Axle Rpm', 'Left Exhaust Temperature',
       'Right Exhaust Temperature', 'Left Rear Parking Brake Oil Pressure',
       'Right Rear Parking Brake Oil Pressure', 'Parking Brake',
       'Brake Pump Pressure', 'Engine Oil Pressure', 'Actual Gear',
       'Ground Speed', 'Engine Speed', 'Transmission Input Speed',
       'Shift Lever Position', 'Throttle Position',
       'Percent Engine Load At Current Engine Speed', 'Truck Payload',
       'Transmission Output Speed', 'Transmission Output Speed #1',
       'Transmission Output Speed #2', 'Air Filter #1 Restriction ',
       'Air Filter #4 Restriction ', 'Air Filter #3 Restriction',
       'Air Filter #2 Restriction ', 'Tank Fuel Level',
       'Percent Fuel Position', 'Fuel Rail Pressure',
       'Cargando', 'Viaje Descargado', 'Viaje Cargado', 'Descarga']]  # Reemplaza 'feature1', 'feature2', 'feature3' con tus nombres de características
y = datos_equipo_32['Fuel Consumption Rate'] # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo

# Añadir constante a las características (intercepto)
X = sm.add_constant(X)

# Crear el modelo de regresión lineal con statsmodels
modelo = sm.OLS(y, X).fit()

# Imprimir un resumen estadistico del modelo
print(modelo.summary())

### 4.2 Ver Correlaciones entre mis variables

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado 'datos'

# Lista de variables que deseas analizar
variables_a_analizar = ['x', 'y', 'z', 
        #'eq_id', 
       'id_trabaj_t',
       'tramosidsnew_t', 
        #'Status Truck', '
        'Left Rear Axle Rpm',
       'Right Rear Axle Rpm', 'Left Exhaust Temperature',
       'Right Exhaust Temperature', 'Left Rear Parking Brake Oil Pressure',
       'Right Rear Parking Brake Oil Pressure', 'Parking Brake',
       'Brake Pump Pressure', 'Engine Oil Pressure', 'Actual Gear',
       'Ground Speed', 'Engine Speed', 'Transmission Input Speed',
       'Shift Lever Position', 'Throttle Position',
       'Percent Engine Load At Current Engine Speed', 'Truck Payload',
       'Transmission Output Speed', 'Transmission Output Speed #1',
       'Transmission Output Speed #2', 'Air Filter #1 Restriction ',
       'Air Filter #4 Restriction ', 'Air Filter #3 Restriction',
       'Air Filter #2 Restriction ', 'Tank Fuel Level',
       'Percent Fuel Position', 'Fuel Rail Pressure',
       'Cargando', 'Viaje Descargado', 'Viaje Cargado', 'Descarga', 'Fuel Consumption Rate']

# Filtrar el DataFrame para incluir solo las variables de interés
datos_subset = datos_equipo_32[variables_a_analizar]

# Crear una matriz de correlación para el subconjunto de variables
matriz_correlacion = datos_subset.corr()

# Configurar el estilo de Seaborn
sns.set(style="whitegrid")

# Aumentar el tamaño del gráfico para mostrar más detalles
plt.figure(figsize=(25, 8))  #ancho * altura

# Crear un mapa de calor (heatmap) de la matriz de correlación con desplazamiento habilitado
ax = sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap="coolwarm", linewidths=.5)

# Agregar etiquetas a los ejes X e Y
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment='right', fontsize=8)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje x)
ax.tick_params(axis='x', labelsize=12)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje y)
ax.tick_params(axis='y', labelsize=12)

plt.title("Matriz de Correlación Variables")
plt.show()

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado 'datos'

# Lista de variables que deseas analizar
variables_a_analizar = [
        'Left Exhaust Temperature',
        'Right Exhaust Temperature',  
        'Parking Brake',
        'Engine Speed', 
        'Shift Lever Position', 'Throttle Position',
        'Percent Engine Load At Current Engine Speed',
         'Air Filter #1 Restriction ',
         'Air Filter #4 Restriction ', 'Air Filter #3 Restriction',
         'Air Filter #2 Restriction ', 
        'Percent Fuel Position', 'Fuel Rail Pressure','Fuel Consumption Rate']

# Filtrar el DataFrame para incluir solo las variables de interés
datos_subset = datos_equipo_32[variables_a_analizar]

# Crear una matriz de correlación para el subconjunto de variables
matriz_correlacion = datos_subset.corr()

# Configurar el estilo de Seaborn
sns.set(style="whitegrid")

# Aumentar el tamaño del gráfico para mostrar más detalles
plt.figure(figsize=(25, 8))  #ancho * altura

# Crear un mapa de calor (heatmap) de la matriz de correlación con desplazamiento habilitado
ax = sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap="coolwarm", linewidths=.5)

# Agregar etiquetas a los ejes X e Y
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment='right', fontsize=8)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje x)
ax.tick_params(axis='x', labelsize=12)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje y)
ax.tick_params(axis='y', labelsize=12)

plt.title("Matriz de Correlación Variables")
plt.show()

### 4.3 Ver la varianza de mis variables

In [0]:
datos_varianza=datos_equipo_32[['x', 'y', 'z', 
        #'eq_id', 
       'id_trabaj_t',
       'tramosidsnew_t', 
        #'Status Truck', '
        'Left Rear Axle Rpm',
       'Right Rear Axle Rpm', 'Left Exhaust Temperature',
       'Right Exhaust Temperature', 'Left Rear Parking Brake Oil Pressure',
       'Right Rear Parking Brake Oil Pressure', 'Parking Brake',
       'Brake Pump Pressure', 'Engine Oil Pressure', 'Actual Gear',
       'Ground Speed', 'Engine Speed', 'Transmission Input Speed',
       'Shift Lever Position', 'Throttle Position',
       'Percent Engine Load At Current Engine Speed', 'Truck Payload',
       'Transmission Output Speed', 'Transmission Output Speed #1',
       'Transmission Output Speed #2', 'Air Filter #1 Restriction ',
       'Air Filter #4 Restriction ', 'Air Filter #3 Restriction',
       'Air Filter #2 Restriction ', 'Tank Fuel Level',
       'Percent Fuel Position', 'Fuel Rail Pressure',
       'Cargando', 'Viaje Descargado', 'Viaje Cargado', 'Descarga']]

In [0]:
desviacion_estandar_sorted

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calcular la varianza de cada columna
desviacion_estandar = np.sqrt(datos_varianza.var())  #desviacion estandar 

# Redondear la varianza a 2 decimales
#variance_rounded = variance.round(2)

# Ordenar las varianzas de mayor a menor
desviacion_estandar_sorted = desviacion_estandar.sort_values(ascending=False)

# Graficar los resultados
plt.figure(figsize=(10, 6))
desviacion_estandar_sorted.plot(kind='bar')
plt.title('Varianza de Variables del DataFrame')
plt.xlabel('Variables')
plt.ylabel('Varianza')
# plt.xticks(rotation=45)
plt.show()

Variables clave para incluir en el modelo serían:
- Fuel Rail Pressure (Presión del riel de combustible)
- Truck Payload (Carga del camión)
- Engine Speed (Velocidad del motor)
- Throttle Position (Posición del acelerador)
- Percent Engine Load (Carga del motor al % de la velocidad actual)
- Ground Speed (Velocidad en tierra)
- Percent Fuel Position (porcentaje de la inyección de combustible)
- Tank Fuel Level (Nivel del tanque de combustible) **

## 5. Analisis de Series Temporales

#### 5.1 Serie Temporal con datos Originales

In [0]:
import matplotlib.pyplot as plt
 
# Set pyplot style
# Plot
datos_equipo_28_enero['Fuel Consumption Rate'].plot(title='Consumo Combustible - Time Plot', figsize=(10,6)) # datos_equipo_32['Fuel Consumption Rate'],indica que los valores de esta columna irán en el eje Y
plt.ylabel('Consumo Combustible (Galones)')   # X está implícito y toma los valores del índice del DataFrame
plt.xlabel('Date')

#### 5.2 Gráfico de líneas donde el eje X representa las fechas y el eje Y el consumo de combustible en galones

Metodos para Eliminar Ruido de Datos

1. Suavizado con Media Móvil

Por ejemplo si tenemos un consumo por dia, Consumo Diario = [10, 12, 14, 13, 15, 17, 20, 18, 16, 14]
- Y decides usar una ventana de 3 días para el suavizado.
- Resultados:

El primer valor de la serie suavizada será 12 (promedio de los primeros 3 días).
El segundo valor será 13 (promedio de los días 2, 3 y 4).
Y así sucesivamente.

In [0]:
import plotly.graph_objs as go
import plotly.express as px

# Suavizado con media móvil
window_size = 30  # Tamaño de la ventana de la media móvil (puedes ajustar esto según tus datos)
datos_equipo_28_enero['Smoothed Fuel Consumption Rate'] = datos_equipo_28_enero['Fuel Consumption Rate'].rolling(window=window_size).mean()

# Crear la figura interactiva con Plotly
fig = go.Figure()

# Agregar la línea del consumo de combustible suavizado
fig.add_trace(go.Scatter(
    x=datos_equipo_28_enero.index,  # Eje X: Fechas
    y=datos_equipo_28_enero['Smoothed Fuel Consumption Rate'],  # Eje Y: Consumo suavizado
    mode='lines',
    name='Media Móvil',
    line=dict(color='orange')
))

# Crear los ticks manualmente cada 3 días
tick_values = pd.date_range(start=datos_equipo_28_enero.index.min(), end=datos_equipo_28_enero.index.max(), freq='2D')

# Configurar los títulos y etiquetas de los ejes
fig.update_layout(
    title='Consumo Combustible - Time Plot',
    xaxis_title='Fecha',
    yaxis_title='Consumo Combustible (Galones)',
    xaxis=dict(
        tickformat='%Y-%m-%d',  # Formato para las fechas
        tickangle=45,  # Rotar etiquetas del eje X
        tickvals=tick_values,  # Colocar etiquetas cada 3 días
        #dtick='D3'  # Mostrar cada 7 días en el eje X
    ),
    autosize=False,
    width=1000,
    height=600
)

# Mostrar el gráfico interactivo
fig.show()

### 5.3 Analisis de Tendencia, Estacionalidad, Residuo

In [0]:
print(datos_equipo_28_enero.index.freq) # Mostrar la frecuencia establecida

In [0]:
import pandas as pd

# Suponiendo que tu índice es un DatetimeIndex
# Asegúrate de que el índice esté en formato DatetimeIndex
datos_equipo_28_enero_sin_duplicados.index = pd.to_datetime(datos_equipo_28_enero_sin_duplicados.index)

# Reindexar para establecer una frecuencia de 2 minutos
datos_reindexados = datos_equipo_28_enero_sin_duplicados.resample('2T').mean()

# Mostrar las primeras filas del DataFrame reindexado
print(datos_reindexados.head())

In [0]:
# Interpolación para llenar valores faltantes
datos_interpolados = datos_reindexados.interpolate(method='linear')

# Alternativamente, puedes usar el método de llenado hacia adelante o hacia atrás
# datos_interpolados = datos_reindexados.fillna(method='ffill')  # Llenar hacia adelante
# datos_interpolados = datos_reindexados.fillna(method='bfill')  # Llenar hacia atrás

# Mostrar las primeras filas del DataFrame con interpolación
print(datos_interpolados.head())

In [0]:
# Calcular las diferencias entre fechas consecutivas
intervalos = datos_equipo_28_enero.index.to_series().diff().dropna()

# Mostrar las primeras diferencias para ver los intervalos
intervalos.value_counts()

In [0]:
# Calcular las diferencias entre fechas consecutivas
intervalos3 = datos_filtros.index.to_series().diff().dropna()

# Mostrar las primeras diferencias para ver los intervalos
intervalos3.value_counts()

In [0]:
# Calcular las diferencias entre fechas consecutivas
intervalos2 = datos_equipo_28_enero_sin_duplicados.index.to_series().diff().dropna()

# Mostrar las primeras diferencias para ver los intervalos
intervalos2.value_counts()

In [0]:
# Eliminar duplicados conservando solo la primera ocurrencia
datos_equipo_28_enero_sin_duplicados = datos_equipo_28_enero[~datos_equipo_28_enero.index.duplicated(keep='first')]

# Mostrar las primeras filas del DataFrame limpio
datos_equipo_28_enero_sin_duplicados.head()

In [0]:
# Calcular las diferencias entre marcas de tiempo consecutivas
diferencias = datos_equipo_28_enero_sin_duplicados.index.to_series().diff()

# Filtrar solo las diferencias de exactamente 2 minutos (120 segundos)
diferencia_2_minutos = pd.Timedelta(minutes=2)

# Seleccionar las filas donde la diferencia entre las marcas de tiempo es exactamente 2 minutos
datos_filtros = datos_equipo_28_enero_sin_duplicados[diferencias == diferencia_2_minutos]

# También puede que desees incluir el primer registro, que no tiene diferencia de tiempo con el anterior
datos_filtros = pd.concat([datos_filtros, datos_equipo_28_enero_sin_duplicados.iloc[[0]]])

# Mostrar las primeras filas del DataFrame filtrado
datos_filtros.head()

In [0]:
datos_equipo_28_enero.shape,datos_equipo_28_enero_sin_duplicados.shape

In [0]:
datos_filtros.index

In [0]:
datos_equipo_32 = datos_equipo_32.asfreq('2T')  # Ajusta '2T' según la frecuencia real de tus datos

In [0]:
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt


# Asegúrate de que el índice es de tipo DatetimeIndex
# datos_equipo_32.index = pd.to_datetime(datos_equipo_32.index)

window_size = 15

# Define el período de estacionalidad (por ejemplo, si los datos son diarios y la estacionalidad es semanal, el período sería 7)
seasonal_period = 2 # Ajusta este valor según la estacionalidad de tus datos

# Descomposición STL
stl = STL(datos_equipo_32['Fuel Consumption Rate'], period=seasonal_period, seasonal=window_size)
result = stl.fit()

# Graficar la tendencia, estacionalidad y residuos
plt.figure(figsize=(12,8))
plt.subplot(3,1,1)
plt.plot(result.trend, label='Tendencia')
plt.legend()
plt.subplot(3,1,2)
plt.plot(result.seasonal, label='Estacionalidad')
plt.legend()
plt.subplot(3,1,3)
plt.plot(result.resid, label='Residuo')
plt.legend()
plt.tight_layout()
plt.show()

### 5.4 Analisis Autoregresivo de la Serie Temporal

In [0]:
# Hacer un gráfico de autocorrelación para la serie de consumo de combustible
plot_acf(datos_equipo_32['Fuel Consumption Rate'], lags=40)  # 'lags=40' son los retardos
plt.title('Autocorrelación del Consumo de Combustible')
plt.show()

In [0]:
!pip list

In [0]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
from langchain.document_loaders import WebBaseLoader